In [ ]:
import time

import os
import sys
import tempfile

import numpy as np
import pandas as pd
import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

def check_patch_in_bounds(x, y, X_dim, Y_dim):
    """ Usage: TrueFalse = check_patch_in_bounds(x, y, X_dim, Y_dim)
                determine if the box is within the image
    Args:
        x:               a tuple, list or array (x_start, x_end)
        y:               a tuple, list or array (y_start, Y_end)
        X_dim:           a tuple, list or array (Image_X_start, Image_X_end)
        Y_dim:           a tuple, list or array (Image_Y_start, Image_Y_end)
    """
    if x[0] > x[1] or y[0] > y[1] or X_dim[0] > X_dim[1] or Y_dim[0] > Y_dim[1]:
        return False
    
    if x[0] >= X_dim[0] and y[0] >= Y_dim[0] and x[1] < X_dim[1] and y[1] < Y_dim[1]:
        return True
    
    else:
        return False

    
def im_pair_hori(im_0, im_1):
    """ Usage: new_im = im_pair_hori(im_0, im_1)
            combine a list of PIL images horizontaly
    """
    w0 = im_0.size[0]
    w = w0 + im_1.size[0]
    h = max(im_0.size[1], im_1.size[1])

    new_im = tip.Image.new('RGB', (w, h) )
    box = (0, 0, w0, h)
    new_im.paste(im_0, box)
    
    box = (w0, 0, w, h)
    new_im.paste(im_1, box)

    return new_im

def get_patch_pairs_array(run_parameters):
    """ Usage: patch_pair_array = get_patch_pairs_array(run_parameters)
    
    Returns:
        patch_pairs_array:  list of tuples = [  (x0, y0, x1, y1),
                                                (x0, y0, x1, y1), ... , (x0, y0, x1, y1) ]
    """
    
    float_wsi = run_parameters['wsi_floatname']
    offset_x = run_parameters['offset_x']
    offset_y = run_parameters['offset_y']
    
    patch_pair_array = []
    
    patch_location_array = get_patch_location_array_for_image_level(run_parameters)
    
    fixed_levels_dict = get_level_sizes_dict(fixed_wsi)
    fixed_max_width = fixed_levels_dict['image_size'][0]
    fixed_max_height = fixed_levels_dict['image_size'][1]

    float_levels_dict = get_level_sizes_dict(float_wsi)
    float_max_width = float_levels_dict['image_size'][0]
    float_max_height = float_levels_dict['image_size'][1]

    last_patch_number = len(patch_location_array) - 1
    patch_number = 0
    while patch_number < last_patch_number:
        patch_number += 1
        y0, x0 = patch_location_array[patch_number]
        x1 = x0 - offset_x
        y1 = y0 - offset_y

        big_enough = x0 >= 0 and y0 >= 0 and x1 >= 0 and y1 >= 0
        fixed_small_enough = x0 < fixed_max_width and y0 < fixed_max_height 
        float_small_enough =  x1 < float_max_width and y1 < float_max_height

        if big_enough and fixed_small_enough and float_small_enough:
            patch_pair_array.append((x0, y0, x1, y1))
    
    return patch_pair_array

In [ ]:
#                                              Locate the Data, assign & normalize local names
test_data_dir = '../../DigiPath_MLTK_data/RegistrationDevData/'
offset_data_file = os.path.join(test_data_dir, 'wsi_pair_sample.csv')
fixed_wsi = os.path.join(test_data_dir, '54742d6c5d704efa8f0814456453573a.tiff')
float_wsi = os.path.join(test_data_dir, 'e39a8d60a56844d695e9579bce8f0335.tiff')

#                                              Read the offset csv
if os.path.isfile(offset_data_file):
    offset_df = pd.read_csv(offset_data_file)

offset_x = int(round(offset_df['truth_offset_x'].iloc[0]))
offset_y = int(round(offset_df['truth_offset_y'].iloc[0]))

auto_x = int(round(offset_df['auto_offset_x'].iloc[0]))
auto_y = int(round(offset_df['auto_offset_y'].iloc[0]))

#                                              Read the images metadata
fixed_levels_dict = get_level_sizes_dict(fixed_wsi)
fixed_max_width = fixed_levels_dict['image_size'][0]
fixed_max_height = fixed_levels_dict['image_size'][1]
    
float_levels_dict = get_level_sizes_dict(float_wsi)
float_max_width = float_levels_dict['image_size'][0]
float_max_height = float_levels_dict['image_size'][1]

#                                              Display the input data
# offset csv
print('\n%25s: %s'%('Offset file', 'wsi_pair_sample.csv'))
print(offset_df.iloc[0], '\n\n%25s:'%('run_parameters'))
#                                              assign & display run_parameters
run_parameters = {'wsi_filename': fixed_wsi, 
                  'wsi_floatname': float_wsi,
                  'thumbnail_divisor': 20, 
                  'patch_select_method': 'threshold_rgb2lab', 
                  'rgb2lab_threshold': 80, 
                  'image_level': 0, 
                  'patch_height': 224, 
                  'patch_width': 224, 
                  'threshold': 0, 
                  'offset_x': offset_x, 
                  'offset_y': offset_y, 
                  'patch_stride_fraction': 1.0,
                  'float_offset_x': -1 * offset_x, 
                  'float_offset_y': -1 * offset_y}
# run_parameters
for k, v in run_parameters.items():
    print('%25s: %s'%(k,v))
# fixed image metadata
print('\n%25s: %s'%('fixed_image input', '54742d6c5d704efa8f0814456453573a.tiff'))
for k, v in fixed_levels_dict.items():
    print('%25s: %s'%(k,v))
# float image metadata
print('\n%25s: %s'%('float_image input','e39a8d60a56844d695e9579bce8f0335.tiff'))
for k, v in float_levels_dict.items():
    print('%25s: %s'%(k,v))

In [ ]:
"""
        get fixed image generator
"""
run_parameters['image_level'] = 3
t0 = time.time()
fxd_im_gen = PatchImageGenerator(run_parameters)
# patch_dict = fxd_im_gen.next_patch()
print('%0.3f'%(time.time()-t0), '%i patches'%(len(fxd_im_gen.image_level_loc_array)))

In [ ]:
"""
        Open the float file
"""
wsi_obj_0 = openslide.OpenSlide(run_parameters['wsi_floatname'])

image_level = run_parameters['image_level']
patch_size = (run_parameters['patch_width'], run_parameters['patch_height'])

X_dim, Y_dim = wsi_obj_0.level_dimensions[image_level]

In [ ]:
"""
        Iterate the fixed, get and show the float image
"""
n_patches = len(fxd_im_gen.image_level_loc_array)
print('%i patches found'%(n_patches))
t_iter = time.time()
n2do = 10 # n_patches + 1

float_offset_x = run_parameters['float_offset_x']
float_offset_y = run_parameters['float_offset_y']
out_of_bounds_count = 0

stop_teration = False
for _ in range(n2do):
    try:
        patch_dict = fxd_im_gen.next_patch()
    except:
        print('Stop Iteration')
        stop_teration = True
        pass
    if stop_teration: 
        break
        
    x1 = patch_dict['level_0_x'] + float_offset_x
    y1 = patch_dict['level_0_y'] + float_offset_y
    
    x_b = [x1, x1+patch_size[0]]
    y_b = [y1, y1+patch_size[1]]

    if check_patch_in_bounds(x_b, y_b, [0, X_dim], [0, Y_dim]) == True:
        print('\n\n')
        print('x', patch_dict['level_0_x'], x1, 'y', patch_dict['level_0_y'], y1)
        flot_im = wsi_obj_0.read_region((x1, y1), image_level, patch_size)
        im_pair = im_pair_hori(patch_dict['patch_image'], flot_im)
        display(im_pair)
    else:
        out_of_bounds_count += 1
        print('out of bounds')
        
print('%0.3f'%(time.time()-t_iter), '%i out of bounds'%(out_of_bounds_count))



In [ ]:
"""
        clean up and close out
"""
try:
    del fxd_im_gen
    print('deleted')
    wsi_obj_0.close()
    print('closed')
except:
    print('already deleted')
    pass

In [ ]:
run_file_dir = '../data/run_files/'
yaml_file = 'wsi_file_to_patches_dir.yml'
run_file = os.path.join(run_file_dir, yaml_file)
run_directory = '../../run_dir'
run_parameters = get_run_parameters(run_directory, yaml_file)
run_parameters['patch_stride_fraction'] = 0.5
run_parameters['offset_x'] = 0
run_parameters['offset_y'] = 0


for k, v in run_parameters.items():
    print('%20s: %s'%(k, v))